In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import catboost as cb
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# # Load data
# train = pd.read_csv('/mnt/sagemaker-nvme/feature_merge/train_CLEAN_FINAL.csv')
# test = pd.read_csv('/mnt/sagemaker-nvme/feature_merge/test_CLEAN_FINAL.csv')

# Also save sample
X_train = pd.read_csv("/mnt/sagemaker-nvme/feature_merge/X_train_reduced.csv")
X_test = pd.read_csv("/mnt/sagemaker-nvme/feature_merge/X_test_reduced.csv")
y_train = pd.read_csv("/mnt/sagemaker-nvme/feature_merge/y_train.csv")

In [ ]:
# print(f"Train: {train.shape} | Test: {test.shape}")
# print(f"\nTarget distribution:")
# print(train['price'].describe())
# # print(f"\nLog_price distribution:")
# # print(train['log_price'].describe())
# # train.drop("log_price", inplace = True, axis=1)

# # Check for problematic prices
# print(f"\n⚠️  Prices < 1: {(train['price'] < 1).sum()}")
# print(f"⚠️  Prices > 1000: {(train['price'] > 1000).sum()}")

In [ ]:
# def prepare_smape_safe_target(train):
#     """Fix extreme values that break SMAPE"""
#     train = train.copy()

#     # SMAPE breaks with near-zero denominators
#     # Clip very low prices (0.5 minimum to prevent division issues)
#     min_price = 0.5
#     train['price'] = train['price'].clip(lower=min_price)

#     # Winsorize extreme outliers (cap at 99.5th percentile)
#     q995 = train['price'].quantile(0.995)
#     train['price'] = train['price'].clip(upper=q995)

#     # Recreate log target AFTER clipping
#     train['log_price'] = np.log1p(train['price'])

#     print(f"✅ Target range after SMAPE-safe clipping:")
#     print(f"   Price: [{train['price'].min():.2f}, {train['price'].max():.2f}]")
#     print(f"   Log: [{train['log_price'].min():.4f}, {train['log_price'].max():.4f}]")

#     return train

# train = prepare_smape_safe_target(train)

In [ ]:
# def reduce_feature_dimensionality(train, test):
#     """Reduce 2430 → ~200-300 features to prevent overfitting"""

#     # Separate feature groups
#     sample_id = 'sample_id'
#     target_cols = ['price', 'log_price']

#     # Feature groups from your summary
#     vit_cols = [c for c in train.columns if 'vit_emb_' in c]  # 768
#     name_cols = [c for c in train.columns if 'name_emb_' in c]  # 768
#     desc_cols = [c for c in train.columns if 'desc_emb_' in c]  # 768
#     clip_cols = [c for c in train.columns if 'clip_' in c and 'emb' not in c]
#     umap_cols = [c for c in train.columns if 'umap_' in c]  # 50
#     cluster_cols = ['hdbscan_cluster', 'cluster_size', 'cluster_density',
#                    'membership_strength', 'distance_to_center', 'is_noise',
#                    'isolation_score', 'is_small_cluster', 'is_large_cluster']
#     pca_cols = ['pc1', 'pc2']
#     yolo_cols = [c for c in train.columns if 'yolo' in c or 'box_' in c or 'object_' in c]
#     tabular_cols = ['value', 'total_value', 'unit', 'brand_cat', 'brand_tier',
#                    'is_bulk', 'is_premium', 'has_size_descriptor']

#     print(f"📊 Feature breakdown:")
#     print(f"   ViT embeddings: {len(vit_cols)}")
#     print(f"   Name embeddings: {len(name_cols)}")
#     print(f"   Desc embeddings: {len(desc_cols)}")
#     print(f"   UMAP: {len(umap_cols)}")
#     print(f"   Other features: ~{len(train.columns) - len(vit_cols) - len(name_cols) - len(desc_cols)}")

#     # Strategy: Reduce embeddings via SVD, keep engineered features
#     def reduce_embeddings(train_data, test_data, cols, n_components):
#         """Apply TruncatedSVD to reduce embedding dimensions"""
#         if len(cols) == 0:
#             return train_data, test_data, []

#         svd = TruncatedSVD(n_components=n_components, random_state=42)

#         train_reduced = svd.fit_transform(train_data[cols])
#         test_reduced = svd.transform(test_data[cols])

#         new_cols = [f"{cols[0].split('_')[0]}_svd_{i}" for i in range(n_components)]

#         train_df = pd.DataFrame(train_reduced, columns=new_cols, index=train_data.index)
#         test_df = pd.DataFrame(test_reduced, columns=new_cols, index=test_data.index)

#         print(f"   ✅ Reduced {len(cols)} → {n_components} ({svd.explained_variance_ratio_.sum():.3f} variance)")

#         return train_df, test_df, new_cols

#     # Reduce each embedding group
#     print("\n🔧 Reducing embeddings to prevent overfitting:")

#     vit_train, vit_test, vit_new = reduce_embeddings(train, test, vit_cols, 50)
#     name_train, name_test, name_new = reduce_embeddings(train, test, name_cols, 30)
#     desc_train, desc_test, desc_new = reduce_embeddings(train, test, desc_cols, 30)

#     # Combine reduced embeddings + engineered features
#     keep_cols = (umap_cols + cluster_cols + pca_cols + yolo_cols +
#                  tabular_cols + clip_cols)

#     # Filter to existing columns only
#     keep_cols = [c for c in keep_cols if c in train.columns and c in test.columns]

#     print(f"\n   Keeping {len(keep_cols)} engineered features")

#     # Build final datasets
#     X_train = pd.concat([
#         train[keep_cols],
#         vit_train,
#         name_train,
#         desc_train
#     ], axis=1)

#     X_test = pd.concat([
#         test[keep_cols],
#         vit_test,
#         name_test,
#         desc_test
#     ], axis=1)

#     # Fill any remaining NaNs
#     X_train = X_train.fillna(-999)
#     X_test = X_test.fillna(-999)

#     y_train = train['log_price']

#     print(f"\n✅ FINAL FEATURE COUNT: {X_train.shape[1]} (reduced from 2430)")
#     print(f"   Feature-to-sample ratio: 1:{len(X_train) // X_train.shape[1]}")

#     return X_train, X_test, y_train

# X_train, X_test, y_train = reduce_feature_dimensionality(train, test)


In [ ]:
# print("\n💾 Saving reduced features for reproducibility...")

# # Save to local disk (fast)
# X_train.to_csv('/mnt/sagemaker-nvme/feature_merge/X_train_reduced.csv', index=False)
# X_test.to_csv('/mnt/sagemaker-nvme/feature_merge/X_test_reduced.csv', index=False)
# y_train.to_csv('/mnt/sagemaker-nvme/feature_merge/y_train.csv', index=False, header=['log_price'])

# # Also save sample_id for final submission
# train['sample_id'].to_csv('/mnt/sagemaker-nvme/feature_merge/train_sample_ids.csv', index=False, header=['sample_id'])
# test['sample_id'].to_csv('/mnt/sagemaker-nvme/feature_merge/test_sample_ids.csv', index=False, header=['sample_id'])

# print(f"   ✅ Saved X_train: {X_train.shape}")
# print(f"   ✅ Saved X_test: {X_test.shape}")
# print(f"   ✅ Saved y_train: {y_train.shape}")

# # Optional: Upload to S3 (if you have boto3 configured)
# import boto3
# s3 = boto3.client('s3')
# bucket = 'amazon-ml-challenge-yourname'  # Change this!

# try:
#     s3.upload_file('/mnt/sagemaker-nvme/feature_merge/X_train_reduced.csv',
#                    bucket, 'amazon_ml_challenge/X_train_reduced.csv')
#     s3.upload_file('/mnt/sagemaker-nvme/feature_merge/X_test_reduced.csv',
#                    bucket, 'amazon_ml_challenge/X_test_reduced.csv')
#     print(f"   ✅ Uploaded to S3: {bucket}/amazon_ml_challenge/")
# except Exception as e:
#     print(f"   ⚠️  S3 upload skipped: {e}")

In [ ]:
# ==============================
# LIGHTGBM TRAINING - FINAL (CPU)
# ==============================

import lightgbm as lgb
from sklearn.model_selection import KFold
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("\n" + "="*80)
print(" LIGHTGBM TRAINING - CPU MODE (NO MORE GPU ISSUES!) ")
print("="*80)

def lgbm_smape_eval(y_true_log, y_pred_log):
    """Custom SMAPE for LightGBM"""
    y_true = np.expm1(y_true_log)
    y_pred = np.expm1(y_pred_log)
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred))
    smape_score = np.mean(200 * numerator / (denominator + 1e-8))
    return 'smape', smape_score, False

def smape_metric(y_true, y_pred):
    """SMAPE for evaluation - matches lgbm_smape_eval formula"""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred))
    return np.mean(100 * numerator / (denominator + 1e-8))


def train_lgbm_final(X, y, X_test, n_folds=5):
    """LightGBM training - CPU optimized"""

    params = {
        'objective': 'regression_l1',  # MAE for SMAPE
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'max_depth': 6,
        'learning_rate': 0.03,
        'feature_fraction': 0.6,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'min_child_samples': 50,
        'min_split_gain': 0.1,
        'reg_alpha': 1.0,
        'reg_lambda': 3.0,
        'n_estimators': 3000,
        'n_jobs': -1,  # Use all CPU cores
        'verbose': -1,
        'random_state': 42
    }

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(X_test))
    smape_scores = []

    import time
    start_time = time.time()

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        fold_start = time.time()

        print(f"\n{'='*60}")
        print(f"🔥 LightGBM Fold {fold + 1}/{n_folds}")
        print(f"{'='*60}")

        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric=lgbm_smape_eval,
            callbacks=[
                lgb.early_stopping(150, verbose=False),
                lgb.log_evaluation(500)
            ]
        )

        oof_preds[val_idx] = model.predict(X_val)
        test_preds += model.predict(X_test) / n_folds

        # val_smape = smape_metric(np.expm1(y_val), np.expm1(oof_preds[val_idx]))
        val_smape = smape_metric(np.expm1(y_val.values), np.expm1(oof_preds[val_idx]))

        smape_scores.append(val_smape)

        fold_time = (time.time() - fold_start) / 60
        print(f"\n   📊 Fold {fold + 1} SMAPE: {val_smape:.4f}%")
        print(f"   📊 Best iteration: {model.best_iteration_}")
        print(f"   ⏱️  Fold time: {fold_time:.1f} mins")

        # Estimate remaining time
        elapsed = (time.time() - start_time) / 60
        avg_fold_time = elapsed / (fold + 1)
        remaining = avg_fold_time * (n_folds - fold - 1)
        print(f"   ⏱️  Estimated remaining: {remaining:.1f} mins")

    avg_smape = np.mean(smape_scores)
    std_smape = np.std(smape_scores)
    total_time = (time.time() - start_time) / 60

    print(f"\n{'='*60}")
    print(f"✅ LightGBM CV SMAPE: {avg_smape:.4f}% ± {std_smape:.4f}%")
    print(f"⏱️  Total training time: {total_time:.1f} mins")
    print(f"{'='*60}")

    return oof_preds, test_preds, avg_smape, std_smape



In [ ]:
import re

# === FIX COLUMN NAMES (ADD THIS NOW!) ===
print("\n🔧 Cleaning column names for LightGBM...")

def clean_column_names(df):
    """Remove special JSON characters from column names"""
    # Remove all special characters except letters, numbers, and underscores
    new_names = {col: re.sub(r'[^A-Za-z0-9_]+', '', col) for col in df.columns}

    # Handle duplicates (add suffix if name already exists)
    new_n_list = list(new_names.values())
    final_names = {}
    for i, (old_col, new_col) in enumerate(new_names.items()):
        if new_col in new_n_list[:i]:
            final_names[old_col] = f'{new_col}_{i}'
        else:
            final_names[old_col] = new_col

    return df.rename(columns=final_names)

# Clean both train and test
X_train = clean_column_names(X_train)
X_test = clean_column_names(X_test)

print(f"✅ Cleaned {len(X_train.columns)} column names")
print(f"   Sample columns: {list(X_train.columns[:5])}")

# Now training will work!


In [ ]:

# START TRAINING NOW!
lgb_oof, lgb_test, lgb_score, lgb_std = train_lgbm_final(
    X_train, y_train, X_test, n_folds=5
)

In [ ]:
# ==============================
# CATBOOST GPU - USING BUILT-IN SMAPE
# ==============================

import catboost as cb
from sklearn.model_selection import KFold
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("\n" + "="*80)
print(" CATBOOST GPU TRAINING - BUILT-IN SMAPE METRIC ")
print("="*80)

def smape_metric_display(y_true, y_pred):
    """SMAPE for display (0-200 scale to match LightGBM)"""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred))
    return np.mean(100 * numerator / (denominator + 1e-8))

def train_catboost_gpu(X, y, X_test, n_folds=5):
    """CatBoost GPU - using built-in SMAPE"""

    params = {
        'task_type': 'GPU',
        'devices': '0',
        'loss_function': 'MAE',
        'eval_metric': 'SMAPE',
        'iterations': 4000,
        'learning_rate': 0.03,
        'depth': 6,
        'l2_leaf_reg': 5,
        'random_strength': 0.5,
        'bagging_temperature': 0.2,
        'min_data_in_leaf': 50,
        'bootstrap_type': 'Bayesian',
        'early_stopping_rounds': 200,
        'use_best_model': True,
        'random_seed': 42,
        'verbose': 200,
        'allow_writing_files': False
    }


    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(X_test))
    smape_scores = []

    import time
    start_time = time.time()

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        fold_start = time.time()

        print(f"\n{'='*60}")
        print(f"🚀 CatBoost Fold {fold + 1}/{n_folds}")
        print(f"{'='*60}")

        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        train_data = cb.Pool(X_tr, y_tr)
        val_data = cb.Pool(X_val, y_val)

        model = cb.CatBoostRegressor(**params)
        model.fit(
            train_data,
            eval_set=val_data,
            use_best_model=True,
            verbose=False,
            plot=False
        )

        oof_preds[val_idx] = model.predict(X_val)
        test_preds += model.predict(X_test) / n_folds

        # Calculate SMAPE on original scale (0-200 scale)
        val_smape = smape_metric_display(np.expm1(y_val.values), np.expm1(oof_preds[val_idx]))
        smape_scores.append(val_smape)

        fold_time = (time.time() - fold_start) / 60
        print(f"\n   📊 Fold {fold + 1} SMAPE: {val_smape:.4f}%")
        print(f"   📊 Best iteration: {model.get_best_iteration()}")
        print(f"   ⏱️  Fold time: {fold_time:.1f} mins")

        elapsed = (time.time() - start_time) / 60
        avg_fold_time = elapsed / (fold + 1)
        remaining = avg_fold_time * (n_folds - fold - 1)
        print(f"   ⏱️  Estimated remaining: {remaining:.1f} mins")

    avg_smape = np.mean(smape_scores)
    std_smape = np.std(smape_scores)
    total_time = (time.time() - start_time) / 60

    print(f"\n{'='*60}")
    print(f"✅ CatBoost CV SMAPE: {avg_smape:.4f}% ± {std_smape:.4f}%")
    print(f"⏱️  Total training time: {total_time:.1f} mins")
    print(f"{'='*60}")

    return oof_preds, test_preds, avg_smape, std_smape



In [ ]:
# START TRAINING
cat_oof, cat_test, cat_score, cat_std = train_catboost_gpu(
    X_train, y_train, X_test, n_folds=5
)

# Save predictions
np.save('/mnt/sagemaker-nvme/feature_merge/cat_oof.npy', cat_oof)
np.save('/mnt/sagemaker-nvme/feature_merge/cat_test.npy', cat_test)

print(f"\n💾 Saved CatBoost predictions")
print(f"✅ CatBoost training complete!")

In [ ]:
# ==============================
# SIMPLE ENSEMBLE - NO OPTIMIZATION NEEDED
# ==============================

import numpy as np
import pandas as pd

print("\n" + "="*80)
print(" CREATING ENSEMBLE - SIMPLE AVERAGE ")
print("="*80)

# Load predictions
lgb_oof = np.load('/mnt/sagemaker-nvme/feature_merge/lgb_oof.npy')
lgb_test = np.load('/mnt/sagemaker-nvme/feature_merge/lgb_test.npy')
cat_oof = np.load('/mnt/sagemaker-nvme/feature_merge/cat_oof.npy')
cat_test = np.load('/mnt/sagemaker-nvme/feature_merge/cat_test.npy')

print(f"✅ Loaded all predictions")

# SMAPE metric
# def smape_metric(y_true, y_pred):
#     """SMAPE (0-100 scale)"""
#     numerator = np.abs(y_pred - y_true)
#     denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
#     return np.mean(numerator / (denominator + 1e-8)) * 100

# Calculate individual model scores
y_true = y_train.values
# lgb_smape = smape_metric(np.expm1(y_true), np.expm1(lgb_oof))
# cat_smape = smape_metric(np.expm1(y_true), np.expm1(cat_oof))

# print(f"\n📊 Individual Model Scores:")
# print(f"   LightGBM: {lgb_smape:.4f}%")
# print(f"   CatBoost: {cat_smape:.4f}%")

# Simple average ensemble (best for similar-performing models)
ensemble_oof = (lgb_oof + cat_oof) / 2
ensemble_test = (lgb_test + cat_test) / 2

# Calculate ensemble score
# ensemble_smape = smape_metric(np.expm1(y_true), np.expm1(ensemble_oof))

# print(f"\n🏆 Ensemble Score:")
# print(f"   Simple Average: {ensemble_smape:.4f}%")
# print(f"   Improvement: {min(lgb_smape, cat_smape) - ensemble_smape:.4f}%")

# Convert to original scale
final_prices = np.expm1(ensemble_test)

# Clip to reasonable range
min_price = 0.5
max_price = np.expm1(y_true).max() * 1.2
final_prices = np.clip(final_prices, min_price, max_price)

print(f"\n✅ Final predictions:")
print(f"   Price range: [{final_prices.min():.2f}, {final_prices.max():.2f}]")
print(f"   Mean: {final_prices.mean():.2f}")
print(f"   Median: {np.median(final_prices):.2f}")


In [ ]:
# ==============================
# CREATE SUBMISSION
# ==============================

# Create submission DataFrame
submission = pd.DataFrame({
    'sample_id': test['sample_id'].values,
    'price': final_prices
})

# Save submission
submission.to_csv('submission_final.csv', index=False)

print(f"\n💾 SUBMISSION SAVED!")
print(f"   File: submission_final.csv")
print(f"   Rows: {len(submission)}")

# Validation checks
print(f"\n🔍 Submission Validation:")
print(f"   No missing values: {submission['price'].isna().sum() == 0}")
print(f"   No negative prices: {(submission['price'] < 0).sum() == 0}")
print(f"   Correct shape: {submission.shape == (75000, 2)}")

print("\n" + "="*80)
print(" 🎉 READY TO SUBMIT! ")
print("="*80)
print(f"Expected Leaderboard Score: ~{ensemble_smape:.2f}%")
print(f"Improvement vs Baseline (54.38%): {54.38 - ensemble_smape:.2f}%")
print("="*80)
